# Data Preparation

In [1]:
# import all libraries for notebook
import pandas as pd
from pandas import json_normalize
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import json
from json import loads, dumps
from pprint import pprint
import torch
from transformers import pipeline
from tqdm import tqdm

In [2]:
# Import files
df_replies = pd.read_csv('../data/twitter/tweets_isReply.csv', dtype={'id': 'object'}, low_memory=False)
df_tweets = pd.read_csv('../data/twitter/tweets_isTweet.csv', dtype={'id': 'object'}, low_memory=False)
df_all = pd.read_csv('../data/twitter/tweets.csv', dtype={'id': 'object'}, low_memory=False)
# Create and prepare Dataframes
# Original tweets
df_nonQuotedTweets = df_tweets[df_tweets['quoted_tweet'].isna()]
df_nonQuotedTweets = df_nonQuotedTweets.rename(columns={'id': 'tweet_id', 'text': 'tweet_text'})
# Quoted tweets
df_quotedTweets = df_tweets.dropna(subset=['quoted_tweet'])
df_quotedTweets = df_quotedTweets.rename(columns={'id': 'tweet_id', 'text': 'tweet_text'})

In [35]:
print("[df_quotedTweets]")
print("Data from: ", df_quotedTweets['createdAt'].max(), " to: ", df_quotedTweets['createdAt'].min())
print("Shape:", df_quotedTweets.shape)
print("[df_nonQuotedTweets]")
print("Data from: ", df_nonQuotedTweets['createdAt'].max(), " to: ", df_nonQuotedTweets['createdAt'].min())
print("Shape:", df_nonQuotedTweets.shape)
print("[df_all]")
print("Data from: ", df_all['createdAt'].max(), " to: ", df_all['createdAt'].min())
print("Shape:", df_all.shape)


[df_quotedTweets]
Data from:  Wed Sep 27 21:38:13 +0000 2023  to:  Fri Apr 04 02:42:55 +0000 2025
Shape: (8349, 26)
[df_nonQuotedTweets]
Data from:  Wed Sep 27 23:06:04 +0000 2023  to:  Fri Apr 04 04:18:02 +0000 2025
Shape: (3113, 26)
[df_all]
Data from:  Wed Sep 25 20:45:08 +0000 2024  to:  Fri Apr 04 02:22:25 +0000 2025
Shape: (21848, 26)


In [36]:
### Functions
# Removing unnecessary data
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+httpss\S+", '', text, flags=re.MULTILINE) # Remove Url
    text = re.sub(r"\@w+|\#", '', text) # remove @ and #
    text = re.sub(r"[^\w\s]", '', text) # remove punctuation
    text_tokens = text.split()
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

# Reduction of dimensionality by abstracting word to word stem
stemmer = PorterStemmer() 
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

def polarity(text):
    return TextBlob(text).sentiment.polarity

def sentiment(label):
    if label < 0:
        return "negative"
    elif label == 0:
        return "neutral"
    elif label > 0:
        return "positive"
    return None

In [37]:
### Sentiment Analysis with df_quotedTweets
## Prepare data
# Normalize quoted_tweet column
df_quotedTweets_normalized = pd.json_normalize(
    df_quotedTweets['quoted_tweet'].apply(json.loads),
    sep='_'
)

# Link both dataframes by index
df_quotedTweets_normalized.index = df_quotedTweets.index

# Rename columns
df_quotedTweets_normalized.columns = ['quoted_tweet_' + col for col in df_quotedTweets_normalized.columns]

# Concatenate the two dataframes
df_final = pd.concat([
    df_quotedTweets[['tweet_id', 'tweet_text']],
    df_quotedTweets_normalized[['quoted_tweet_id', 'quoted_tweet_text']]
], axis=1)

## Process data and perform analysis
# Pre-processing
df_final.quoted_tweet_text = df_final['quoted_tweet_text'].apply(data_processing)
df_final.tweet_text = df_final['tweet_text'].apply(data_processing)

# Stemming
df_final['quoted_tweet_text'] = df_final['quoted_tweet_text'].apply(lambda x: stemming(x))

# Polarity
df_final['polarity'] = df_final['quoted_tweet_text'].apply(polarity)

# Sentiment
df_final['sentiment'] = df_final['polarity'].apply(sentiment)

df_final

,tweet_id,tweet_text,quoted_tweet_id,quoted_tweet_text,polarity,sentiment
2,1917225430702240067,big deal,1917223651625099407,last week treasury went live first automated p...,-0.005519,negative
4,1917103264417649121,whoa,1917011279757066291,invisible puppet masters ais disturbing new ro...,0.066279,positive
5,1917099777327829386,next week grok 35 early beta release supergrok...,1917011847623987257,grok 3 sends usage soaring 10x spike downloads...,0.066667,positive
6,1917071819003334728,existential crisis,1917059115417014780,friendly reminder make babies japans total fer...,0.075000,positive
9,1917040536378335721,starlink trying service plan commitment exchan...,1917029886432317947,0 standard kit 12month residential service pla...,0.075000,positive
...,...,...,...,...,...,...
11433,1655968899903418373,massive public manipulation,1655968201422012418,legacy media going shove bigotry porn throats ...,0.050000,positive
11434,1655967673107337216,whatsapp cannot trusted,1654856617723301888,whatsapp using microphone background asleep si...,0.000000,neutral
11438,1655661734244261891,livestream starting soon,1655596008246587394,tesla lithium refinery groundbreaking streamed...,0.136364,positive
11447,1654680950163361792,highly recommend organizations sign validate a...,1641596848921276417,verified organizations new way organizations a...,0.123636,positive


In [38]:
### Emotion Analysis with df_nonQuotedTweets
import pandas as pd
import torch
from transformers import pipeline
from tqdm import tqdm

# Load Hugging Face's emotion classifier
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion", top_k=None, device=0 if device == "cuda" else -1)

# Define the six Ekman emotions
ekman_emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]


def truncate_text(text, max_tokens=512):
    words = text.split()  # Split text into words
    return " ".join(words[:max_tokens])  # Keep only the first 512 tokens

def compute_emotions(text):
    if not isinstance(text, str) or text.strip() == "":
        print(f"Invalid text: {text[:50]}...")
        return {emotion: 0.0 for emotion in ekman_emotions}  # Return zero scores for empty text

    try:
        truncated_text = truncate_text(text)  # Truncate text to fit within 512 tokens
        results = classifier(truncated_text)[0]  # Get emotion scores from the model
        emotion_scores = {result['label']: result['score'] for result in results}

        # Extract only the six Ekman emotions and normalize missing ones
        print(f"Emotion scores for text: {text[:50]}...")
        return {emotion: emotion_scores.get(emotion, 0.0) for emotion in ekman_emotions}

    except Exception as e:
        print(f"Error processing text: {text[:50]}... Error: {e}")
        return {emotion: 0.0 for emotion in ekman_emotions}  # Return zero scores on failure

def append_emotions(df, text_column):
    if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' not found in the DataFrame.")

    print("Appending emotion scores to DataFrame...")

    emotion_scores = [compute_emotions(text) for text in tqdm(df[text_column])]
    emotions_df = pd.DataFrame(emotion_scores)
    emotions_df.index = df.index
    emotions_df.columns = [f"{text_column}_emotions_{emotion}" for emotion in emotions_df.columns]
    result_df = pd.concat([df, emotions_df], axis=1)

    return result_df

# Filter dataframe for text to be analyzed
df_nonQuotedTweets = df_nonQuotedTweets[['tweet_id', 'tweet_text']]
df_nonQuotedTweets = append_emotions(df_nonQuotedTweets, text_column='tweet_text')



Device set to use cpu


Appending emotion scores to DataFrame...


  0%|          | 3/3113 [00:01<21:20,  2.43it/s]  

Emotion scores for text: https://t.co/U6tI9pdin6...
Emotion scores for text: https://t.co/1c1WjFpOva...
Emotion scores for text: https://t.co/6xSd8l67FN...
Emotion scores for text: Knock, knock … it’s @DOGE https://t.co/yFDXlZgnmn...


  0%|          | 7/3113 [00:01<08:47,  5.89it/s]

...

Emotion scores for text: https://t.co/g7UiTiTPhI...
Emotion scores for text: https://t.co/KmlrLxZXCo...


100%|█████████▉| 3109/3113 [05:42<00:00,  9.43it/s]

Emotion scores for text: The more screentime users spend looking at a post,...
Emotion scores for text: They are the same picture https://t.co/hGOSqQ78Wl...
Emotion scores for text: Cult / Culture...


100%|██████████| 3113/3113 [05:43<00:00,  9.08it/s]

Emotion scores for text: Worth reading The Machine Stops by EM Forster if y...
Emotion scores for text: Falcon Heavy https://t.co/g6Ix8RLgVS...
Emotion scores for text: Interview with @BillMaher https://t.co/0hpT4Sxzzj...


In [39]:
df_nonQuotedTweets

,tweet_id,tweet_text,tweet_text_emotions_anger,tweet_text_emotions_disgust,tweet_text_emotions_fear,tweet_text_emotions_joy,tweet_text_emotions_sadness,tweet_text_emotions_surprise
0,1917726279195058338,https://t.co/U6tI9pdin6,0.028279,0.0,0.017699,0.907494,0.038398,0.002830
1,1917693698281787564,https://t.co/1c1WjFpOva,0.012809,0.0,0.009525,0.937607,0.029090,0.003167
3,1917114631287718009,https://t.co/6xSd8l67FN,0.013412,0.0,0.015682,0.909002,0.048675,0.004338
7,1917048786633036129,"Knock, knock … it’s @DOGE https://t.co/yFDXlZgnmn",0.310502,0.0,0.027653,0.606090,0.045680,0.003166
8,1917048324215189802,Population collapse is an existential problem\...,0.013613,0.0,0.024462,0.782486,0.174791,0.002312
...,...,...,...,...,...,...,...,...
11457,1653290370048966656,They are the same picture https://t.co/hGOSqQ78Wl,0.011047,0.0,0.005982,0.956148,0.019805,0.001437
11458,1653262203343052806,Cult / Culture,0.793149,0.0,0.061943,0.092314,0.030917,0.003264
11459,1653075645138710528,Worth reading The Machine Stops by EM Forster ...,0.008727,0.0,0.001879,0.982474,0.003021,0.001596
11460,1652911737794883586,Falcon Heavy https://t.co/g6Ix8RLgVS,0.016851,0.0,0.002507,0.971238,0.006077,0.000872
